In [1]:
import pandas as pd

In [ ]:
rename_dict = {
    'Country': 'country',
    'Group': 'group',
    'Type': 'type',
    'Maker/Brand':'brand',
    '2018': 'cars_sold'}

df = (pd.read_html('data/cars_data.html')[0]
      .dropna()
      .rename(columns=rename_dict)
      .astype({'cars_sold': 'int64'}))

In [ ]:
def get_country_brand_df(data: pd.DataFrame) -> pd.DataFrame:
    country_brand = data[~data.brand.str.contains('Total')]
    country_brand = country_brand[~country_brand.group.str.contains('Other')]

    car_types = data.type.unique()
    for car_type in car_types:
        country_brand = country_brand[~country_brand.group.str.contains(car_type, regex=False)]

    country_brand = (
        country_brand
        .loc[:, ['country', 'brand', 'cars_sold']]
        .groupby(['country', 'brand'])
        .sum()
        .drop(index=['Uzbekistan', 'Belarus'])
        .reset_index()
    )

    country_brand = country_brand[country_brand.cars_sold > 100]

    return country_brand


def get_country_total_df(country_brand: pd.DataFrame) -> pd.DataFrame:
    country_total = (
        country_brand
        .loc[:, ['country', 'cars_sold']]
        .groupby('country')
        .sum()
        .rename(columns={'cars_sold': 'country_total'})
        .reset_index()
    )

    return country_total

In [ ]:
country_brand = get_country_brand_df(df)
country_total = get_country_total_df(country_brand)

In [ ]:
car_brands_df = (country_brand.set_index('country')
                 .join(country_total.set_index('country'))
                 .reset_index()
                 .assign(share=lambda x: x.cars_sold / x.country_total)
                 )
car_brands_df = car_brands_df[~car_brands_df.brand.str.contains('OEM')]

In [ ]:
car_brands_df.to_csv('country_brand.csv', index=False)